# Learn pyspark 

In [32]:
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import Imputer

### create virtual environment in conda

open anaconda propmt in windows:

go to the directory  where the files are placed
connand to create venv: conda create -n venv_name
command to activate venv: activate venv_name
install a kernal for jupyter notebook in venv: ipython kernel install --user --name=cenvkernel
open jupyter notebook: jupyter notebook
create a new file in venv kernal by right clicking: new -> kernal_name 

First install pyspark in virtual environment: pip install pyspark / conda install pyspark

In [33]:
CURR_DIR = os.path.abspath('')
DOC_NAME = os.path.join(CURR_DIR, '..', "docs", "employee_dataset.csv")

### Verify first the input file with pandas

In [34]:
df = pd.read_csv(DOC_NAME)
df_describe = df.describe()
print(df_describe)

             Age  Experience        Salary
count  11.000000   11.000000     10.000000
mean   42.454545    5.181818  35190.000000
std    19.054467    2.638870  15749.317446
min    23.000000    2.000000   4500.000000
25%    24.500000    3.500000  27500.000000
50%    36.000000    5.000000  35600.000000
75%    56.000000    6.500000  42800.000000
max    78.000000   10.000000  65000.000000


### Create a spark session and load data

In [35]:
spark = SparkSession.builder.appName('Practise').getOrCreate()
df_pyspark = spark.read.csv(DOC_NAME, header=True, inferSchema=True)
df_pyspark.show()

+----+----+----------+------+
|Name| Age|Experience|Salary|
+----+----+----------+------+
| aaa|  25|         2| 45000|
| bbb|  56|         4| 65000|
| ccc|  24|         5| 35000|
| ddd|  36|         7| 25000|
| eee|  56|         9|  null|
| fff|null|      null| 36200|
| ggg|  36|        10|  null|
| hhh|  78|      null| 35200|
| iii|  65|         5|  4500|
| jjj|  45|         3| 25000|
| kkk|  23|      null|  null|
| lll|null|         2| 45000|
|null|  23|      null| 36000|
|null|null|         4|  null|
|null|null|         6|  null|
+----+----+----------+------+



### Missing data - add
imputer command is used to add data in missing places mean, meadia

In [36]:
imputer = Imputer(
    inputCols = ['Age', 'Experience', 'Salary'],
    outputCols = ["{}_imputed".format(c) for c in ['age', 'Experience','Salary']]
).setStrategy("mean")
imputer.fit(df_pyspark).transform(df_pyspark).show()

+----+----+----------+------+-----------+------------------+--------------+
|Name| Age|Experience|Salary|age_imputed|Experience_imputed|Salary_imputed|
+----+----+----------+------+-----------+------------------+--------------+
| aaa|  25|         2| 45000|         25|                 2|         45000|
| bbb|  56|         4| 65000|         56|                 4|         65000|
| ccc|  24|         5| 35000|         24|                 5|         35000|
| ddd|  36|         7| 25000|         36|                 7|         25000|
| eee|  56|         9|  null|         56|                 9|         35190|
| fff|null|      null| 36200|         42|                 5|         36200|
| ggg|  36|        10|  null|         36|                10|         35190|
| hhh|  78|      null| 35200|         78|                 5|         35200|
| iii|  65|         5|  4500|         65|                 5|          4500|
| jjj|  45|         3| 25000|         45|                 3|         25000|
| kkk|  23| 

### Missing data - drop
option available: how - any, all, threshold - 1, 2 

In [37]:
df_pyspark_drop = df_pyspark.na.drop(how ='any')

### Filter options

In [39]:
df_pyspark_drop.filter("Salary<=45000").show()

+----+---+----------+------+
|Name|Age|Experience|Salary|
+----+---+----------+------+
| aaa| 25|         2| 45000|
| ccc| 24|         5| 35000|
| ddd| 36|         7| 25000|
| iii| 65|         5|  4500|
| jjj| 45|         3| 25000|
+----+---+----------+------+



In [40]:
df_pyspark_drop.filter("Salary<=45000").select(['Name','Age']).show()

+----+---+
|Name|Age|
+----+---+
| aaa| 25|
| ccc| 24|
| ddd| 36|
| iii| 65|
| jjj| 45|
+----+---+



In [42]:
df_pyspark_drop.filter(df_pyspark_drop["Salary"]<=45000).show()

+----+---+----------+------+
|Name|Age|Experience|Salary|
+----+---+----------+------+
| aaa| 25|         2| 45000|
| ccc| 24|         5| 35000|
| ddd| 36|         7| 25000|
| iii| 65|         5|  4500|
| jjj| 45|         3| 25000|
+----+---+----------+------+



mulitple filter condition

In [50]:
df_pyspark_drop.filter((df_pyspark_drop["Salary"]<=45000)  | (df_pyspark_drop["Salary"]>=20000)).show()

+----+---+----------+------+
|Name|Age|Experience|Salary|
+----+---+----------+------+
| aaa| 25|         2| 45000|
| bbb| 56|         4| 65000|
| ccc| 24|         5| 35000|
| ddd| 36|         7| 25000|
| iii| 65|         5|  4500|
| jjj| 45|         3| 25000|
+----+---+----------+------+



inverse or not operator

In [52]:
df_pyspark_drop.filter(~(df_pyspark_drop["Salary"]<=45000)).show()

+----+---+----------+------+
|Name|Age|Experience|Salary|
+----+---+----------+------+
| bbb| 56|         4| 65000|
+----+---+----------+------+

